In [ ]:
# # one-time setup
# # installing cmake
# !wget https://github.com/Kitware/CMake/releases/download/v3.31.4/cmake-3.31.4-linux-x86_64.sh
# !chmod +x cmake-3.31.4-linux-x86_64.sh
# !sudo ./cmake-3.31.4-linux-x86_64.sh --prefix=/usr/local --skip-license 
# !rm cmake-3.31.4-linux-x86_64.sh
# !cmake --version


# # installing cuda
# # !sudo apt remove --purge '^nvidia-.*' -y
# !sudo apt autoremove -y
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu$(lsb_release -rs | tr -d '.')/x86_64/cuda-keyring_1.0-1_all.deb
# !sudo dpkg -i cuda-keyring_1.0-1_all.deb
# # !rm cuda-keyring_1.0-1_all.deb
# !sudo apt update
# !sudo apt install cuda -y
# !echo 'export PATH=/usr/local/cuda/bin:$PATH' >> ~/.bashrc
# !echo 'export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH' >> ~/.bashrc
# !source ~/.bashrc


# # stable_diffusion_cpp_python
# !sudo apt install build-essential g++-11 ninja-build
# !sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 100
# !sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 100
# !CMAKE_ARGS="-DCUDA_ARCH=sm_89 -DSD_CUDA=ON -DSD_FLASH_ATTN=ON" pip install stable-diffusion-cpp-python --upgrade --force-reinstall --no-cache-dir

In [ ]:
!export MAKEFLAGS="-j$(nproc)"
!pip install numpy torch hf_transfer huggingface-hub huggingface_hub[hf_transfer] flash-attn
!pip install transformers accelerate optimum auto-gptq pickleshare safetensors pymongo json5 diffusers protobuf peft hf_transfer supabase \
    bitsandbytes torchvision pytorch_lightning xformers "xfuser[diffusers,flash-attn]" --upgrade

In [ ]:
!pip install python-dotenv supabase

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # This loads the variables from .env into os.environ

import os
from supabase import create_client, Client

url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [ ]:
%%capture
# Setting up HF transfer
from huggingface_hub import login
import base64
t = 'aGZfaHZqck9VTXFvTXF3dW9HR3JoTlZKSWlsZUtFTlNQbXRjTw=='
k = base64.b64decode(t.encode()).decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_HF_TRANSFER=1

In [ ]:
# in terminal pip install hf_transfer
# !huggingface-cli download black-forest-labs/FLUX.1-dev ae.safetensors
!huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors
!huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors
# !huggingface-cli download kudzueye/boreal-flux-dev-v2 boreal-v2.safetensors
# !huggingface-cli download kudzueye/Boreal boreal-flux-dev-lora-v04_1000_steps.safetensors
# !huggingface-cli download Shakker-Labs/FLUX.1-dev-LoRA-AntiBlur FLUX-dev-lora-AntiBlur.safetensors

!huggingface-cli download black-forest-labs/FLUX.1-dev

In [ ]:
# ComfyUI setup
%cd  /workspace/dtback
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /workspace/dtback/ComfyUI
!pip install -r requirements.txt
%cd /workspace/dtback/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager comfyui-manager
!git clone https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git
!git clone https://github.com/AlekPet/ComfyUI_Custom_Nodes_AlekPet.git # for ACE++ cn workflow

!huggingface-cli download black-forest-labs/FLUX.1-dev flux1-dev.safetensors --local-dir /workspace/dtback/ComfyUI/models/unet
!huggingface-cli download black-forest-labs/FLUX.1-Fill-dev flux1-fill-dev.safetensors --local-dir /workspace/dtback/ComfyUI/models/unet

!huggingface-cli download black-forest-labs/FLUX.1-dev ae.safetensors --local-dir /workspace/dtback/ComfyUI/models/vae
!huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors --local-dir /workspace/dtback/ComfyUI/models/clip
!huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors --local-dir /workspace/dtback/ComfyUI/models/clip

%cd /workspace/dtback/ComfyUI/models/loras
!huggingface-cli download kudzueye/Boreal boreal-flux-dev-lora-v04_1000_steps.safetensors --local-dir /workspace/dtback/ComfyUI/models/loras
!huggingface-cli download Shakker-Labs/FLUX.1-dev-LoRA-AntiBlur FLUX-dev-lora-AntiBlur.safetensors --local-dir /workspace/dtback/ComfyUI/models/loras
!huggingface-cli download Shakker-Labs/AWPortrait-FL AWPortrait-FL-lora.safetensors --local-dir /workspace/dtback/ComfyUI/models/loras
!huggingface-cli download ali-vilab/ACE_Plus portrait/comfyui_portrait_lora64.safetensors --local-dir /workspace/dtback/ComfyUI/models/loras # ACE++
!curl -L -J -O "https://drive.google.com/uc?export=download&id=1UNuErS3bvfOUwGqGAPU0AA8hXKEOKKyD"
%cd /workspace/dtback/ComfyUI

# sudo mkdir -p --mode=0755 /usr/share/keyrings
# curl -fsSL https://pkg.cloudflare.com/cloudflare-main.gpg | sudo tee /usr/share/keyrings/cloudflare-main.gpg >/dev/null
# echo 'deb [signed-by=/usr/share/keyrings/cloudflare-main.gpg] https://pkg.cloudflare.com/cloudflared any main' | sudo tee /etc/apt/sources.list.d/cloudflared.list
# sudo apt-get update && sudo apt-get install cloudflared
# cloudflared tunnel --url http://localhost:8000

%cd /workspace/dtback
!python3 ComfyUI/main.py --port 8000

In [ ]:
%%bash
cd /workspace/dtback
git clone https://github.com/ali-vilab/ACE_plus.git
cd ACE_plus
# pip install -r requirements.txt

In [ ]:
%env FLUX_FILL_PATH=hf://black-forest-labs/FLUX.1-Fill-dev
%env PORTRAIT_MODEL_PATH=hf://ali-vilab/ACE_Plus@portrait/comfyui_portrait_lora64.safetensors        
%env SUBJECT_MODEL_PATH=hf://ali-vilab/ACE_Plus@subject/comfyui_subject_lora16.safetensors        
%env LOCAL_MODEL_PATH=hf://ali-vilab/ACE_Plus@local_editing/comfyui_local_lora16.safetensors

In [ ]:
%cd /workspace/dtback/ACE_plus

%env FLUX_FILL_PATH=hf://black-forest-labs/FLUX.1-Fill-dev
%env ACE_PLUS_FFT_MODEL=hf://ali-vilab/ACE_Plus@ace_plus_fft.safetensors
!python3 infer_fft.py
%cd /workspace/dtback

In [ ]:
%cd /workspace/dtback/ACE_plus
%env FLUX_FILL_PATH=hf://black-forest-labs/FLUX.1-Fill-dev
%env ACE_PLUS_FFT_MODEL=hf://ali-vilab/ACE_Plus@ace_plus_fft.safetensors

import argparse, importlib, io, os, sys

from PIL import Image
from scepter.modules.transform.io import pillow_convert
from scepter.modules.utils.config import Config
from scepter.modules.utils.file_system import FS

if os.path.exists('__init__.py'):
    package_name = 'scepter_ext'
    spec = importlib.util.spec_from_file_location(package_name, '__init__.py')
    package = importlib.util.module_from_spec(spec)
    sys.modules[package_name] = package
    spec.loader.exec_module(package)

from inference.registry import INFERENCES
fs_list = [
    Config(cfg_dict={"NAME": "HuggingfaceFs", "TEMP_DIR": "./cache"}, load=False),
    Config(cfg_dict={"NAME": "ModelscopeFs", "TEMP_DIR": "./cache"}, load=False),
    Config(cfg_dict={"NAME": "HttpFs", "TEMP_DIR": "./cache"}, load=False),
    Config(cfg_dict={"NAME": "LocalFs", "TEMP_DIR": "./cache"}, load=False),
]

for one_fs in fs_list:
    FS.init_fs_client(one_fs)


def run_one_case(pipe,
                input_image = None,
                input_mask = None,
                input_reference_image = None,
                save_path = "examples/output/example.png",
                instruction = "",
                output_h = 1024,
                output_w = 1024,
                seed = -1,
                sample_steps = None,
                guide_scale = None,
                repainting_scale = None,
                use_change=True,
                keep_pixels=True,
                keep_pixels_rate=0.8,
                **kwargs):
    if input_image is not None:
        input_image = Image.open(io.BytesIO(FS.get_object(input_image)))
        input_image = pillow_convert(input_image, "RGB")
    if input_mask is not None:
        input_mask = Image.open(io.BytesIO(FS.get_object(input_mask)))
        input_mask = pillow_convert(input_mask, "L")
    if input_reference_image is not None:
        input_reference_image = Image.open(io.BytesIO(FS.get_object(input_reference_image)))
        input_reference_image = pillow_convert(input_reference_image, "RGB")
    print(repainting_scale)
    image, _, _, _, seed = pipe(
        reference_image=input_reference_image,
        edit_image=input_image,
        edit_mask=input_mask,
        prompt=instruction,
        output_height=output_h,
        output_width=output_w,
        sampler='flow_euler',
        sample_steps=sample_steps or pipe.input.get("sample_steps", 30),
        guide_scale=guide_scale or pipe.input.get("guide_scale", 50),
        seed=seed,
        repainting_scale=repainting_scale,
        use_change=use_change,
        keep_pixels=keep_pixels,
        keep_pixels_rate=keep_pixels_rate
    )
    with FS.put_to(save_path) as local_path:
        image.save(local_path)
    return local_path, seed


model_cfg = Config(load=True, cfg_file="config/ace_plus_fft.yaml")
pipe = INFERENCES.build(model_cfg)
params = {
    "output_h": 1024,
    "output_w": 1024,
    "sample_steps": 30,
    "guide_scale": None,
}
def run():
    module_name = "examples.examples"
    if module_name in sys.modules:
        del sys.modules[module_name]
    mod = importlib.import_module(module_name)
    all_examples = mod.fft_examples
    importlib.reload(mod)
    for example in all_examples:
        example.update(params)
        local_path, seed = run_one_case(pipe, **example)

%cd /workspace/dtback

In [ ]:
%cd /workspace/dtback/ACE_plus
run()
%cd /workspace/dtback

In [ ]:
%env FLUX_FILL_PATH=hf://black-forest-labs/FLUX.1-Fill-dev
%env PORTRAIT_MODEL_PATH=hf://ali-vilab/ACE_Plus@portrait/comfyui_portrait_lora64.safetensors        
%env SUBJECT_MODEL_PATH=hf://ali-vilab/ACE_Plus@subject/comfyui_subject_lora16.safetensors        
%env LOCAL_MODEL_PATH=hf://ali-vilab/ACE_Plus@local_editing/comfyui_local_lora16.safetensors
%cd /workspace/dtback/ACE_plus
!sed -i 's/SAMPLE_STEPS: 28/SAMPLE_STEPS: 30/' ./config/ace_plus_diffusers_infer.yaml
!sed -i 's/assert cfg.args.task_type.upper() in task_model_cfg//g' infer_lora.py

dir = "/workspace/dtback/img"
!mkdir -p {dir}

instruction = """
Keep face absolutely the same.
Photo of a young woman riding a bike
""".replace("'", "")

!python3 infer_lora.py --input_reference_image {dir}/reference.jpg --seed 42 --save_path {dir}/out.jpg --instruction '{instruction}' --output_h 1024 --output_w 1024 \
    --repainting_scale 0.7 --task_type portrait --cfg_folder ./config --infer_type diffusers --guide_scale 40

%cd /workspace/dtback

In [ ]:
%%bash

curl -s https://raw.githubusercontent.com/sivel/speedtest-cli/master/speedtest.py | python3 -
export MAKEFLAGS="-j$(nproc)"
pip install numpy torch
pip install flash-attn --no-build-isolation
pip install transformers accelerate optimum auto-gptq huggingface-hub huggingface_hub[hf_transfer] pickleshare safetensors \
                pymongo json5 diffusers protobuf peft hf_transfer supabase bitsandbytes torchvision --upgrade
pip install xformers "xfuser[diffusers,flash-attn]"
pip install bitsandbytes torchvision
pip install --upgrade transformers diffusers huggingface_hub peft
pip install --force-reinstall -v "triton==3.1.0"

k=$(echo "aGZfaHZqck9VTXFvTXF3dW9HR3JoTlZKSWlsZUtFTlNQbXRjTw==" | base64 --decode)
# Log in to Hugging Face (this assumes Python and huggingface_hub are installed)
python3 -c "from huggingface_hub import login; login(token='$k', add_to_git_credential=False)"
export HUGGINGFACEHUB_API_TOKEN="$k"
export HF_TOKEN="$k"
export HF_HUB_ENABLE_HF_TRANSFER=1


huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors
huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors
huggingface-cli download black-forest-labs/FLUX.1-dev


In [ ]:
# # # generate the base
!huggingface-cli download black-forest-labs/FLUX.1-dev

%cd /workspace/dtback/models/img/loras/
!curl -L -J -O "https://drive.google.com/uc?export=download&id=1UNuErS3bvfOUwGqGAPU0AA8hXKEOKKyD"
%cd /workspace/dtback/

import torch
from diffusers import FluxPipeline

if "pipe" not in globals():
    pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", device_map="balanced", torch_dtype=torch.bfloat16)
    pipe.load_lora_weights("kudzueye/Boreal", weight_name="boreal-flux-dev-lora-v04_1000_steps.safetensors", adapter_name="boreal")
    pipe.load_lora_weights("/workspace/dtback/models/img/loras", weight_name="SameFace_Fix.safetensors", adapter_name="sameface")
    pipe.set_adapters(["boreal", "sameface"], [0.7, -0.3])
    pipe.fuse_lora()
base_prompt = """"
close photo of a fat indian woman in her 40's with huge bust and hips on solid plain gray background wearing corporate clothes.
The woman is facing the camera directly
"""
base_prompt = """
Portrait shot photo of a 24-year-old Korean trainer from San Francisco on a solid plain gray background.
She wears a yellow tank top that fits her comfortably, showing her toned arms and slight collarbones.
Her skin is smooth and evenly toned with a natural dry glow.
There is a soft light on her high cheekbones and a few tiny freckles across the bridge of her nose.
Her face has a clear, oval shape with a defined yet gentle jawline.
Fine lines on her forehead and around her eyes are very subtle, hinting at small expressions of life without any harsh marks.
Her dark brown hair is straight and cut just to shoulder length, with a small side part that lets a few strands fall neatly across her face.
The hair has a slight shine and frames her face evenly. Her dark brown eyes are almond-shaped and bright, with long lashes that add a natural definition.
The eyes are set evenly on her face, and her neatly trimmed eyebrows follow a natural arch that gives a clear outline above her eyes.
Her small, straight nose ends in a soft, rounded tip that fits well with the rest of her features.
Her lips are naturally pink, kept closed in a slight smile, and her small chin is gently rounded.
Her ears are small and sit close to her head, while her slender neck adds to the overall balanced look of her profile.
"""
img = pipe(prompt=base_prompt, guidance_scale=2, height=1024, width=1024, num_inference_steps=40,
           generator=torch.Generator("cuda").manual_seed(10)).images[0]
img.show()
img.save('prebase.webp')

In [ ]:
# flux plain
import torch
from diffusers import FluxPipeline

if "pipe" not in globals():
    pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", device_map="balanced", torch_dtype=torch.bfloat16)
prompt = "photo of a cat"
img = pipe(prompt=prompt, guidance_scale=2, height=1024, width=1024, num_inference_steps=40,
           generator=torch.Generator("cuda").manual_seed(10)).images[0]
img.show()
img.save('prebase.webp')

In [ ]:
# # # remove bg from the base
# https://github.com/PeterL1n/BackgroundMattingV2

try:
    del pipe
except NameError:
    pass
import gc, torch
gc.collect()
torch.cuda.empty_cache()

%cd /workspace/dtback/models/img/
!pip install gdown  -q
import gdown
gdown.download('https://drive.google.com/uc?id=1-t9SO--H4WmP7wUl1tVNNeDkq47hjbv4', 'torchscript_resnet50_fp32.pth', quiet=False)
%cd /workspace/dtback/

import torch
from torchvision.transforms.functional import to_tensor, to_pil_image
from PIL import Image

device = torch.device('cuda')
precision = torch.float32

model = torch.jit.load('/workspace/dtback/models/img/torchscript_resnet50_fp32.pth')
model.backbone_scale = 0.2
model.refine_mode = 'thresholding'
model.refine_threshold = 0.0001
model = model.to(device).eval()

src = Image.open('prebase.webp').convert('RGB')
bgr = Image.new("RGB", src.size, (170, 170, 170)) 

src_tensor = to_tensor(src).unsqueeze(0).to(device, dtype=precision)
bgr_tensor = to_tensor(bgr).unsqueeze(0).to(device, dtype=precision)

pha, fgr = model(src_tensor, bgr_tensor)[:2]

rgba = torch.cat([fgr, pha], dim=1)
result_img = to_pil_image(rgba.squeeze(0).clamp(0, 1), mode='RGBA')
result_img.show()
result_img.save('base.png')

In [ ]:
%%bash

cd /workspace/dtback
git clone -b sd3 https://github.com/kohya-ss/sd-scripts.git
cd sd-scripts
pip install --upgrade -r requirements.txt
mkdir -p loras
rm -rf dataset
mkdir -p dataset
cd dataset
# cp /workspace/dtback/base.png ./base.png
# curl -O https://i.ibb.co/dwccwKCZ/watson.png # 1024
# curl -O https://i.ibb.co/35PNsSrg/watson-512.png # 512
cd ..

cat > dataset.toml << EOF
[[datasets]]
resolution = 1024
batch_size = 8
keep_tokens = 1

  [[datasets.subsets]]
  image_dir = '/workspace/dtback/sd-scripts/dataset'
  class_tokens = 'woman'
  num_repeats = 5
  flip_aug = true
  face_crop_aug_range=[1.0, 1.4]
EOF

cd /workspace/dtback

# create dataset of base on the different backgrounds
python << 'EOF'
import os
from PIL import Image
base_path = "base.png"
output_folder = "/workspace/dtback/sd-scripts/dataset"
os.makedirs(output_folder, exist_ok=True)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),
          (0, 255, 255), (255, 128, 0), (128, 0, 128), (128, 128, 128), (255, 255, 255)]
base = Image.open(base_path).convert("RGBA")
for i, (r, g, b) in enumerate(colors):
    bg = Image.new("RGBA", base.size, (r, g, b, 255))
    out = Image.alpha_composite(bg, base)
    out.save(os.path.join(output_folder, f"char_bg_{i}.png"))
EOF


cat > /root/.cache/huggingface/accelerate/default_config.yaml << EOF
compute_environment: LOCAL_MACHINE
debug: false
distributed_type: 'NO'
downcast_bf16: 'no'
dynamo_config:
  dynamo_backend: TENSORRT
enable_cpu_affinity: true
gpu_ids: all
machine_rank: 0
main_training_function: main
mixed_precision: bf16
num_machines: 1
num_processes: 1
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false
EOF

pip install bitsandbytes torchvision
pip install --upgrade transformers diffusers huggingface_hub peft
pip install --force-reinstall -v "triton==3.1.0"

In [ ]:
%%bash

fpath="/root/.cache/huggingface/hub/models--black-forest-labs--FLUX.1-dev/snapshots/0ef5fff789c832c5c7f4e127f94c8b54bbcced44"
epath="/root/.cache/huggingface/hub/models--comfyanonymous--flux_text_encoders/snapshots/6af2a98e3f615bdfa612fbd85da93d1ed5f69ef5"

accelerate launch --mixed_precision bf16 --num_cpu_threads_per_process 10 sd-scripts/flux_train_network.py --pretrained_model_name_or_path "${fpath}/flux1-dev.safetensors" \
--clip_l "${epath}/clip_l.safetensors" --t5xxl "${epath}/t5xxl_fp16.safetensors" --ae "${fpath}/ae.safetensors" --cache_latents_to_disk --save_model_as safetensors --sdpa \
--persistent_data_loader_workers --max_data_loader_n_workers 10 --seed 0 --gradient_checkpointing --mixed_precision bf16 --save_precision bf16 --network_module networks.lora_flux \
--network_dim 8 --network_train_unet_only --optimizer_type adamw8bit --learning_rate 1e-4 --cache_text_encoder_outputs --highvram --max_train_epochs 4 --save_every_n_epochs 1 \
--dataset_config sd-scripts/dataset.toml --output_dir sd-scripts/loras --output_name wm --timestep_sampling shift --discrete_flow_shift 3.1582 --model_prediction_type raw \
--guidance_scale 0.0

In [ ]:
# test the lora
import torch
from diffusers import FluxPipeline

if "pipe" not in globals():
    pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", device_map="balanced", torch_dtype=torch.bfloat16)
    pipe.load_lora_weights("/workspace/dtback/sd-scripts/loras", weight_name="wm.safetensors", adapter_name="wm")
    pipe.load_lora_weights("kudzueye/Boreal", weight_name="boreal-flux-dev-lora-v04_1000_steps.safetensors", adapter_name="boreal")
    pipe.set_adapters(["boreal", "wm"], [0.0, 0.8])
    pipe.fuse_lora()

prompts = [
    "selfie photo of a woman at the Miami beach standing in water with wet hair",
    "photo of a crying woman with a bouquet of flowers from her groom on her wedding",
    "photo of a woman showering",
    "photo of a happy woman playing with her dog",
    "photo of a woman dining out in a fancy restraunt",
    "photo of a woman relaxing on a sofa with a magasine",
    "photo of a woman making funny face on her backyard",
]

for i, prompt in enumerate(prompts):
    img = pipe(
        prompt=prompt, 
        guidance_scale=0, 
        height=1024, 
        width=1024, 
        num_inference_steps=20,
        generator=torch.Generator("cuda").manual_seed(1000)
    ).images[0]

    img.show()

In [ ]:
%%bash
k=$(echo "aGZfaHZqck9VTXFvTXF3dW9HR3JoTlZKSWlsZUtFTlNQbXRjTw==" | base64 --decode)
# Log in to Hugging Face (this assumes Python and huggingface_hub are installed)
python3 -c "from huggingface_hub import login; login(token='$k', add_to_git_credential=False)"
export HUGGINGFACEHUB_API_TOKEN="$k"
export HF_TOKEN="$k"
export HF_HUB_ENABLE_HF_TRANSFER=1

In [ ]:
!echo "aGZfaHZqck9VTXFvTXF3dW9HR3JoTlZKSWlsZUtFTlNQbXRjTw==" | base64 --decode

In [ ]:
%cd /workspace/dtback/
!git clone -b dev https://github.com/camenduru/PuLID-FLUX-hf
%cd PuLID-FLUX-hf
!pip install apex gradio diffusers gradio fire insightface onnx onnxruntime-gpu onnxruntime accelerate timm SentencePiece git+https://github.com/XPixelGroup/BasicSR ftfy einops facexlib fire
!wget https://huggingface.co/camenduru/PuLID/resolve/main/bpe_simple_vocab_16e6.txt.gz -O /workspace/dtback/PuLID-FLUX-hf/eva_clip/bpe_simple_vocab_16e6.txt.gz
!python app.py
%cd /workspace/dtback/

In [ ]:
import torch
import torch.nn.functional as F
from diffusers import FluxPriorReduxPipeline, FluxPipeline
from diffusers.utils import load_image

if "pipe" not in globals():
    pipe_prior_redux = FluxPriorReduxPipeline.from_pretrained("black-forest-labs/FLUX.1-Redux-dev", torch_dtype=torch.bfloat16).to("cuda")
    pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16).to("cuda")

image = load_image("./base.webp")

pipe_prior_output = pipe_prior_redux(image)
img_prompt_embeds = pipe_prior_output["prompt_embeds"]

text_prompt = "woman sitting on an porch"
text_prompt_embeds, pooled_text_embeds, _ = pipe.encode_prompt(
    prompt=text_prompt,
    prompt_2=text_prompt,
    device="cuda",
    num_images_per_prompt=1,
)

# Interpolate the text embeddings to match the image embeddings' sequence length.
target_length = img_prompt_embeds.shape[1]  # e.g. 1241
if text_prompt_embeds.shape[1] != target_length:
    # Transpose to (B, embed_dim, seq_length)
    text_prompt_embeds = text_prompt_embeds.transpose(1, 2)
    # Interpolate along the sequence dimension.
    text_prompt_embeds = F.interpolate(text_prompt_embeds, size=target_length, mode='linear', align_corners=False)
    # Transpose back to (B, seq_length, embed_dim)
    text_prompt_embeds = text_prompt_embeds.transpose(1, 2)

# Blend the embeddings.
alpha = 0.83
blended_embeds = alpha * text_prompt_embeds + (1 - alpha) * img_prompt_embeds

# For pooled embeddings, you might choose to use one source directly or blend if they are compatible.
# Here, we simply use the pooled text embeddings.
blended_pooled = pooled_text_embeds

# Now call the Flux pipeline using the blended embeddings.
img = pipe(
    guidance_scale=0,
    num_inference_steps=20,
    generator=torch.Generator("cuda").manual_seed(0),
    prompt_embeds=blended_embeds,
    pooled_prompt_embeds=blended_pooled,
    **{k: v for k, v in pipe_prior_output.items() if k not in ["prompt_embeds", "pooled_prompt_embeds"]}
).images[0]

img.show()
img.save("redux_blended.webp")


In [ ]:
# generate the rest
import torch
from diffusers import FluxImg2ImgPipeline
from diffusers.utils import load_image

if "pipe" not in globals():
    pipe = FluxImg2ImgPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16).to("cuda")
    pipe.load_lora_weights("kudzueye/Boreal", weight_name="boreal-flux-dev-lora-v04_1000_steps.safetensors", adapter_name="boreal")
    pipe.set_adapters("boreal", 0.15)
    pipe.fuse_lora()


init_image = load_image("./1920_0.6.webp")
width, height = init_image.size
prompt = "photo of a woman in a snowy city"

img = pipe(
    prompt=prompt, image=init_image, num_inference_steps=60, strength=0.6, guidance_scale=7.0,
    generator=torch.Generator("cuda").manual_seed(1000), width=width, height=height,
).images[0]
img.show()
img.save('img.webp')

In [ ]:
# !rm -rf ~/.cache/huggingface
# !rm -rf mongo_persistence && mkdir -p mongo_persistence

In [ ]:
# Downloading models

# !huggingface-cli download mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated-GGUF meta-llama-3.1-8b-instruct-abliterated.Q8_0.gguf --local-dir ~/models/llms
# !huggingface-cli download mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated-GGUF meta-llama-3.1-8b-instruct-abliterated.Q4_K_M.gguf --local-dir ~/models/llms
# !huggingface-cli download mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated-GGUF meta-llama-3.1-8b-instruct-abliterated.Q2_K.gguf --local-dir ~/models/llms

# !huggingface-cli download bartowski/Llama-3.3-70B-Instruct-abliterated-GGUF Llama-3.3-70B-Instruct-abliterated-IQ2_S.gguf --local-dir ~/models/llms

# !huggingface-cli download mradermacher/Qwen2.5-32B-Instruct-abliterated-i1-GGUF Qwen2.5-32B-Instruct-abliterated.i1-Q4_K_S.gguf --local-dir ~/models/llms
# !huggingface-cli download mradermacher/Qwen2.5-32B-Instruct-abliterated-i1-GGUF Qwen2.5-32B-Instruct-abliterated.i1-IQ3_M.gguf --local-dir ~/models/llms

# !huggingface-cli download mradermacher/DeepSeek-R1-Distill-Qwen-32B-abliterated-i1-GGUF DeepSeek-R1-Distill-Qwen-32B-abliterated.i1-Q4_K_M.gguf --local-dir ~/models/llms

In [ ]:
# Setting up LLama.cpp

# !sudo apt install libcurl4-openssl-dev
# !apt-cache policy nvidia-cuda-toolkit
# !sudo apt remove nvidia-cuda-toolkit && sudo apt autoremove

# !git clone https://github.com/ggerganov/llama.cpp --depth 1 --single-branch

# %cd /teamspace/studios/this_studio/llama.cpp
# !cmake -B build -DGGML_CUDA=ON -DGGML_CUDA_F16=ON -DGGML_CUDA_FA_ALL_QUANTS=ON
# !cmake --build build --config Release -j 32
# %cd ~

In [ ]:
# testing llama.cpp tools

# !.~/llama.cpp/build/bin/llama-cli
%cd ~/llama.cpp/build/bin

# !./llama-cli -m ~/llama.gguf -h
# !./llama-cli -m ~/models/llama_6.gguf --gpu-layers 18 --ctx-size 131072 --batch-size 65536 -p "I just " --predict 1024 -no-cnv

!./llama-server -m ~/models/llama_4.gguf --port 8000 --gpu-layers 33 --ctx-size 65536 --batch-size 65536

# !./llama-bench -m ~/models/llama_4.gguf

# !./llama-cli -m ~/models/llama_4.gguf --gpu-layers 33 --ctx-size 94208 --batch-size 65536 -p "Sometimes " --predict 128 -no-cnv

In [ ]:
# short test generation
print(requests.post("http://127.0.0.1:8000/v1/chat/completions", json={"messages": [
    {
      "role": "system",
      "content": "You are a professional sexual romance writer.",
    },
    {
      "role": "user",
      "content": "make up a very explicit sexual scene",          
    }
]}).json()['choices'][0]['message']['content'])

In [ ]:
""" server start command
~/llama.cpp/build/bin/llama-server -m ~/models/llms/llama_8.gguf --port 8000 --gpu-layers 33 --ctx-size 131072 \
--cache-type-k q8_0 --cache-type-v q8_0 --flash-attn
"""

""" start with grammar
~/llama.cpp/build/bin/llama-server -m ~/models/llms/llama_4.gguf --port 8000 --gpu-layers 33 --ctx-size 131072 \
--cache-type-k q4_0 --cache-type-v q4_0 --flash-attn --grammar-file ~/generation/1_bots_text.gbnf
~/llama.cpp/build/bin/llama-server -m ~/models/llms/llama_4.gguf --port 8000 --gpu-layers 33 --ctx-size 131072 \
--cache-type-k q4_0 --cache-type-v q4_0 --flash-attn --grammar-file ~/generation/2_add_img_prompts_to_bots.gbnf
"""

In [ ]:
# installing docker

# !sudo apt install postgresql postgresql-contrib # installing postgres
# !sudo systemctl enable postgresql
# !sudo systemctl start postgresql

# !sudo apt install -y apt-transport-https ca-certificates curl software-properties-common
# !yes | curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
# !sudo add-apt-repository \"deb [arch=amd64] https://download.docker.com/linux/ubuntu focal stable\"
# !apt-cache policy docker-ce
# !sudo apt install docker-ce
# !sudo systemctl status docker

In [ ]:
# start mongo
!wget -qO- https://www.mongodb.org/static/pgp/server-8.0.asc | sudo tee /etc/apt/trusted.gpg.d/server-8.0.asc
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/8.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-8.0.list
!sudo apt-get update
!sudo apt-get install -y mongodb-mongosh

!docker run --name mongodb -p 27017:27017 -v ~/mongo_persistence:/data/db -d mongo
# mongosh --port 27017
# docker stop mongodb && docker rm mongodb

In [ ]:
## grammar builder https://grammar.intrinsiclabs.ai
# !python3 generation/1_bots_text.py

In [ ]:
# stable-diffusion.cpp

# %cd ~
# !rm -rf stable-diffusion.cpp
# !git clone --recursive https://github.com/leejet/stable-diffusion.cpp
# %cd ~/stable-diffusion.cpp
# !git pull origin master
# !git submodule init
# !git submodule update
# %cd ~

# !huggingface-cli download city96/FLUX.1-dev-gguf flux1-dev-Q8_0.gguf --local-dir ./models/img

# !huggingface-cli download black-forest-labs/FLUX.1-dev ae.safetensors --local-dir ./models/img
# !huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors --local-dir ./models/img
# !huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors --local-dir ./models/img
!huggingface-cli download kudzueye/boreal-flux-dev-v2 boreal-v2.safetensors --local-dir ./models/img/loras
!huggingface-cli download kudzueye/Boreal boreal-flux-dev-lora-v04_1000_steps.safetensors --local-dir ./models/img/loras
!mv ./models/img/loras/boreal-flux-dev-lora-v04_1000_steps.safetensors ./models/img/loras/boreal.safetensors
!huggingface-cli download Shakker-Labs/FLUX.1-dev-LoRA-AntiBlur FLUX-dev-lora-AntiBlur.safetensors --local-dir ./models/img/loras
!mv ./models/img/loras/FLUX-dev-lora-AntiBlur.safetensors ./models/img/loras/antiblur.safetensors

# !wget https://github.com/Kitware/CMake/releases/download/v3.31.4/cmake-3.31.4-linux-x86_64.sh
# !chmod +x cmake-3.31.4-linux-x86_64.sh
# !sudo ./cmake-3.31.4-linux-x86_64.sh --prefix=/usr/local --skip-license 
# !rm cmake-3.31.4-linux-x86_64.sh
# !cmake --version

# !mkdir -p ~/stable-diffusion.cpp/build
# %cd ~/stable-diffusion.cpp/build
# !cmake .. -DSD_CUDA=ON
# !cmake --build . --config Release

# quantizing
# !~/stable-diffusion.cpp/build/bin/sd --mode "convert" --type "q8_0" --model ~/models/img/temp/nr.safetensors

import time
start = time.time()

# prompt = """
# mirror selfie photo of a smiling young redhead woman [<lora:boreal:0.3>|<lora:boreal-v2:0.15>|<lora:antilbur:2>]
# """
prompt = """
selfie photo of a smiling young redhead woman<lora:boreal:1>
"""

# %cd ~/models/img
# !~/stable-diffusion.cpp/build/bin/sd --diffusion-model flux1-dev-Q8_0.gguf --vae ae.safetensors --clip_l clip_l.safetensors \
# --t5xxl t5xxl_fp16.safetensors --steps 20 --cfg-scale 1.0 --sampling-method euler -v --diffusion-fa -W 1024 -H 1280 --seed 42 -p "{prompt}" --type q8_0 \
# --lora-model-dir ./loras --output ~/output.jpg --threads 16

# print(f'took {time.time() - start} seconds')

# %cd ~
# from PIL import Image

# with Image.open("output.png") as img:
#     img.convert("RGB").save("output.jpg", "JPEG")
#     !rm output.png
    
# with Image.open("output.png") as img:
#     img.show()

prompt = """
Night photo of a tall Mediterranean woman with long curly dark hair and a curvy body riding a bike at Times Square
    
[<lora:boreal:0.7>|<lora:boreal-v2:0.15>|<lora:antilbur:2>]
"""


# %cd ~/models/img
# !~/stable-diffusion.cpp/build/bin/sd --mode img2img --diffusion-model flux1-dev-Q8_0.gguf --vae ae.safetensors --clip_l clip_l.safetensors \
# --t5xxl t5xxl_fp16.safetensors --steps 20 --cfg-scale 1.0 --sampling-method euler -v --diffusion-fa -W 1024 -H 1280 --seed 42 -p "{prompt}" --type q8_0 \
# --lora-model-dir ./loras \
# --output ~/output_img.jpg --threads 16 --init-img ~/output.jpg --strength 0.9

In [ ]:
from stable_diffusion_cpp import StableDiffusion
import time

base_path = "./models/img"
start = time.time()
flux = StableDiffusion(
    diffusion_model_path=f"{base_path}/flux1-dev-Q8_0.gguf",
    clip_l_path=f"{base_path}/clip_l.safetensors",
    t5xxl_path=f"{base_path}/t5xxl_fp16.safetensors",
    vae_path=f"{base_path}/ae.safetensors",
    # vae_decode_only=True, # True for txt2img
    diffusion_flash_attn=True,
    lora_model_dir=f"{base_path}/loras/",
)
print(f'\nloading took {time.time() - start}')

In [ ]:
# txt2img
%cd ~
import time
from PIL import Image

start = time.time()
output = flux.txt_to_img(
    prompt="""
    
    Photo of a young redhead woman with white clear skin,
    beautiful Irish features,
    field of tulips in the background, dusk, fireflies in the air
    
    [<lora:boreal:0.5>|<lora:antilbur:5>]""",
    sample_steps=15,
    width=768,
    height=1024,
    seed=1000,
    cfg_scale=0.7,
    guidance=5,
    sample_method="euler",
)
delta = round(time.time() - start)
output[0].convert('RGB').save("output.jpg", "JPEG")
from IPython.display import clear_output
clear_output()
print(f'took {delta} seconds')
with Image.open("output.jpg") as img:
    img.show()

In [ ]:
# img2img
%cd ~
import time
from PIL import Image
width, height = Image.open("output.jpg").size
start = time.time()
output = flux.img_to_img(
    prompt="""
    
    Creepshot of a tall Mediterranean woman with long curly dark hair and a curvy body dining out in a fancy restaurant wearing small black dress.
    
    [<lora:boreal:0.7>|<lora:boreal-v2:0.15>|<lora:antilbur:2>]""",
    negative_prompt="worst quality, smooth",
    image="output.jpg", # The input image will be automatically resized to the match the width and height arguments (default: 512x512)
    strength=1,
    sample_steps=20,
    width=width,
    height=height,
    seed=1000,
    cfg_scale=1,
    sample_method="euler",
)
delta = round(time.time() - start)
output[0].convert('RGB').save("output_img.jpg")
from IPython.display import clear_output
clear_output()
print(f'took {delta} seconds')
output[0].show()